In [17]:
!mkdir -p /content/FakeContentDetector/data

!wget https://www.cs.ucsb.edu/~william/data/liar_dataset.zip -P /content/FakeContentDetector/data

!unzip -o /content/FakeContentDetector/data/liar_dataset.zip -d /content/FakeContentDetector/data


--2025-07-03 05:06:57--  https://www.cs.ucsb.edu/~william/data/liar_dataset.zip
Resolving www.cs.ucsb.edu (www.cs.ucsb.edu)... 23.185.0.253, 2620:12a:8000::253, 2620:12a:8001::253
Connecting to www.cs.ucsb.edu (www.cs.ucsb.edu)|23.185.0.253|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://sites.cs.ucsb.edu/~william/data/liar_dataset.zip [following]
--2025-07-03 05:06:57--  https://sites.cs.ucsb.edu/~william/data/liar_dataset.zip
Resolving sites.cs.ucsb.edu (sites.cs.ucsb.edu)... 128.111.27.164
Connecting to sites.cs.ucsb.edu (sites.cs.ucsb.edu)|128.111.27.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1013571 (990K) [application/zip]
Saving to: ‘/content/FakeContentDetector/data/liar_dataset.zip.1’

liar_dataset.zip.1  100%[===================>] 989.82K  --.-KB/s    in 0.1s    

2025-07-03 05:06:57 (7.91 MB/s) - ‘/content/FakeContentDetector/data/liar_dataset.zip.1’ saved [1013571/1013571]

Archive:  /cont

In [18]:
import pandas as pd

df = pd.read_csv('/content/FakeContentDetector/data/train.tsv', sep='\t', header=None)


print(df.shape)
df.head()


(10240, 14)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [20]:
df.columns = [
    'id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state', 'party',
    'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts',
    'pants_on_fire_counts', 'context'
]

df_clean = df[['statement', 'label']].dropna()


In [23]:

real_labels = ['true', 'mostly-true']
fake_labels = ['false', 'pants-fire']

df_clean = df_clean[df_clean['label'].isin(real_labels + fake_labels)]

df_clean['label_binary'] = df_clean['label'].apply(lambda x: 1 if x in real_labels else 0)

df_clean['label_binary'].value_counts()


,count
label_binary,
1,3638
0,2834


In [24]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

tokens = tokenizer(
    list(df_clean['statement']),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='np'
)


In [25]:
labels = df_clean['label_binary'].values


In [26]:
from sklearn.model_selection import train_test_split
import torch

X_train_ids, X_test_ids, X_train_mask, X_test_mask, y_train, y_test = train_test_split(
    tokens['input_ids'],
    tokens['attention_mask'],
    labels,
    test_size=0.1,
    random_state=42
)

X_train_ids = torch.tensor(X_train_ids)
X_train_mask = torch.tensor(X_train_mask)
y_train = torch.tensor(y_train, dtype=torch.long)

X_test_ids = torch.tensor(X_test_ids)
X_test_mask = torch.tensor(X_test_mask)
y_test = torch.tensor(y_test, dtype=torch.long)


In [27]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train_ids, X_train_mask, y_train)
test_dataset = TensorDataset(X_test_ids, X_test_mask, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)


In [28]:
from transformers import DistilBertForSequenceClassification
from torch.optim import AdamW

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
from tqdm import tqdm

model.train()
epochs = 3

for epoch in range(epochs):
    print(f"\n🔁 Epoch {epoch+1}/{epochs}")
    total_loss = 0
    loop = tqdm(train_loader, leave=True)

    for batch in loop:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

    print(f"✅ Average Loss for Epoch {epoch+1}: {total_loss/len(train_loader):.4f}")



🔁 Epoch 1/3


Epoch 1: 100%|██████████| 364/364 [00:59<00:00,  6.07it/s, loss=0.486]


✅ Average Loss for Epoch 1: 0.6518

🔁 Epoch 2/3


Epoch 2: 100%|██████████| 364/364 [01:02<00:00,  5.84it/s, loss=0.689]


✅ Average Loss for Epoch 2: 0.5552

🔁 Epoch 3/3


Epoch 3: 100%|██████████| 364/364 [01:01<00:00,  5.90it/s, loss=0.342]

✅ Average Loss for Epoch 3: 0.3621


In [31]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
  for bathc in test_loader:
    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    labels = batch[2].to(device)

    outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
    )

    logits = outputs.logits
    preds = torch.argmax(logits, dim=1)

    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
report = classification_report(all_labels, all_preds)

print(f"Accurace: {accuracy:.4f}")
print(f"F1 Score: {f1: .4f}")
print(f"Classification Report:\n",report)


Accurace: 0.8750
F1 Score:  0.9091
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.67      0.80       246
           1       0.83      1.00      0.91       410

    accuracy                           0.88       656
   macro avg       0.92      0.83      0.85       656
weighted avg       0.90      0.88      0.87       656



In [32]:
model.save_pretrained("/content/FakeContentDetector/models/text_model")
tokenizer.save_pretrained("/content/FakeContentDetector/models/text_model")

('/content/FakeContentDetector/models/text_model/tokenizer_config.json',
 '/content/FakeContentDetector/models/text_model/special_tokens_map.json',
 '/content/FakeContentDetector/models/text_model/vocab.txt',
 '/content/FakeContentDetector/models/text_model/added_tokens.json',
 '/content/FakeContentDetector/models/text_model/tokenizer.json')

In [33]:
%cd /content/FakeContentDetector


/content/FakeContentDetector


In [37]:
%cd /content

!rm -rf FakeContentDetector

!git clone https://github.com/alfredd25/FakeContentDetector.git

%cd FakeContentDetector


/content
Cloning into 'FakeContentDetector'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 19 (delta 4), reused 8 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 5.32 KiB | 1.06 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/FakeContentDetector


In [82]:
!git config --global user.email "alfredbiju7@gmail.com"
!git config --global user.name "alfredd25"


In [40]:
import os
from getpass import getpass

token = getpass('🔒 Enter your GitHub Personal Access Token (PAT): ')
os.environ['GITHUB_TOKEN'] = token


🔒 Enter your GitHub Personal Access Token (PAT): ··········


In [42]:
!git clone https://github.com/alfredd25/FakeContentDetector.git
%cd FakeContentDetector


Cloning into 'FakeContentDetector'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
Receiving objects: 100% (19/19), 5.32 KiB | 5.32 MiB/s, done.
Resolving deltas: 100% (4/4), done.
remote: Total 19 (delta 4), reused 8 (delta 1), pack-reused 0 (from 0)
/content/FakeContentDetector/FakeContentDetector


In [52]:
%cd /content/FakeContentDetector

/content/FakeContentDetector


In [53]:
!ls notebooks/


0_setup.ipynb  requirements.txt


In [54]:
# Save the model weights
torch.save(model.state_dict(), "models/fake_detector.pt")


In [79]:
%cd /content/FakeContentDetector/FakeContentDetector
!ls -l

/content/FakeContentDetector/FakeContentDetector
total 36
drwxr-xr-x 2 root root 4096 Jul  3 05:34 app
drwxr-xr-x 2 root root 4096 Jul  3 05:34 backend
drwxr-xr-x 2 root root 4096 Jul  3 05:34 data
drwxr-xr-x 2 root root 4096 Jul  3 05:34 frontend
-rw-r--r-- 1 root root 1068 Jul  3 05:34 LICENSE
drwxr-xr-x 2 root root 4096 Jul  3 05:34 models
drwxr-xr-x 2 root root 4096 Jul  3 05:34 notebooks
-rw-r--r-- 1 root root   21 Jul  3 05:34 README.md
-rw-r--r-- 1 root root    0 Jul  3 05:34 requirements.txt
drwxr-xr-x 2 root root 4096 Jul  3 05:34 utils


In [87]:
!find /content/drive -name "1A_Dataset_Collection.ipynb"


find: ‘/content/drive’: No such file or directory
